In [3]:
# 파일을 읽기 모드로 열기
with open("C:\\Users\\lee\\Desktop\\bybit\\api.txt", "r") as file:
    f = []
    # 파일의 각 줄에 대해 반복
    for line in file:
        # 각 줄 출력
        f.append(line.strip())  # strip()은 줄 바꿈 문자(\n) 등을 제거하여 출력합니다.

In [25]:
from pybit.unified_trading import HTTP
from datetime import datetime
import math

now = datetime.now()
now_timestamp = math.floor(now.timestamp()*1000)


session = HTTP(testnet=True)
price = session.get_mark_price_kline(
    category="linear",
    symbol="BTCUSDT",
    interval=1,
    start=now_timestamp-180000,
    end=now_timestamp,
    limit=3,
)

now_time = price['time']
print(price)

{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSDT', 'category': 'linear', 'list': [['1700557680000', '37396', '37412.08', '37396', '37412.08'], ['1700557620000', '37402.4', '37403.04', '37394.8', '37396'], ['1700557560000', '37410.4', '37410.5', '37402.4', '37402.4']]}, 'retExtInfo': {}, 'time': 1700557709219}


In [26]:
from datetime import datetime
unix_timestamp = 1700557680000 / 1000
print(datetime.utcfromtimestamp(unix_timestamp))

2023-11-21 09:08:00


In [32]:
# EMA(Exponential Moving Average, 지수이동평균)

def calculate_ema(data, alpha):
    ema_values = [data[0]]  # 초기값은 첫 번째 데이터로 설정

    for i in range(1, len(data)):
        ema = (1 - alpha) * ema_values[-1] + alpha * data[i]
        ema_values.append(ema)

    return ema_values

# 예시 데이터와 평활 매개변수 설정
data = [5, 8, 9, 12, 7, 10, 15]
alpha = 2 / (1 + 7)  # 7은 변수화하기

# 함수 호출하여 지수이동평균 계산
ema_result = calculate_ema(data, alpha)

# 결과 출력
print("입력 데이터:", data)
print("평활 매개변수 (alpha):", alpha)
print("지수이동평균 결과:", ema_result)


입력 데이터: [5, 8, 9, 12, 7, 10, 15]
평활 매개변수 (alpha): 0.25
지수이동평균 결과: [5, 5.75, 6.5625, 7.921875, 7.69140625, 8.2685546875, 9.951416015625]


In [33]:
# RSI (Relative Strength Index, 상대강도지수)
def calculate_rsi(data, period=14):
    # 주가의 변동 계산
    price_diff = [data[i] - data[i - 1] for i in range(1, len(data))]
    
    # 상승 가격 및 하락 가격 계산
    gain = [diff if diff > 0 else 0 for diff in price_diff]
    loss = [abs(diff) if diff < 0 else 0 for diff in price_diff]
    
    # 평균 상승 가격 및 평균 하락 가격 계산
    avg_gain = sum(gain[:period]) / period
    avg_loss = sum(loss[:period]) / period
    
    # 초기 RSI 계산
    if avg_loss == 0:
        rs = 1  # 0으로 나누는 것을 방지하기 위해 예외 처리
    else:
        rs = avg_gain / avg_loss
    
    rsi_values = [100 - (100 / (1 + rs))]  # 초기 RSI 값
    
    # # 이후 RSI 계산
    # for i in range(period, len(data)):
    #     avg_gain = (avg_gain * (period - 1) + gain[i - 1]) / period
    #     avg_loss = (avg_loss * (period - 1) + loss[i - 1]) / period
        
    #     if avg_loss == 0:
    #         rs = 1
    #     else:
    #         rs = avg_gain / avg_loss
        
    #     rsi = 100 - (100 / (1 + rs))
    #     rsi_values.append(rsi)
    
    return rsi_values

# 예시 데이터 설정
stock_prices = [55.05, 54.90, 54.91, 54.55, 54.41, 54.24, 54.76, 54.55, 54.70, 54.56]

# 함수 호출하여 RSI 계산
rsi_result = calculate_rsi(stock_prices)

# 결과 출력
print("주가 데이터:", stock_prices)
print("RSI 결과:", rsi_result)



주가 데이터: [55.05, 54.9, 54.91, 54.55, 54.41, 54.24, 54.76, 54.55, 54.7, 54.56]
RSI 결과: [36.75675675675685]


In [7]:
# ADX (Average Directioinal Movement Index )

import pandas as pd

def calculate_adx(high, low, close, period=14):
    # True Range 계산
    tr = pd.DataFrame({
        'hl': high - low,
        'hpc': abs(high - close.shift(1)),
        'lpc': abs(low - close.shift(1))
    }).max(axis=1)

    # Directional Movement Index 계산
    plus_dm = (high - high.shift(1)).apply(lambda x: x if x > 0 else 0)
    minus_dm = (low.shift(1) - low).apply(lambda x: x if x > 0 else 0)

    # Average True Range (ATR) 계산
    atr = tr.ewm(span=period, adjust=False).mean()

    # Directional Movement Index (DX) 계산
    plus_di = (plus_dm / atr).ewm(span=period, adjust=False).mean() * 100
    minus_di = (minus_dm / atr).ewm(span=period, adjust=False).mean() * 100

    # ADX 계산
    dx = abs(plus_di - minus_di) / (plus_di + minus_di) * 100
    adx = dx.ewm(span=period, adjust=False).mean()

    return adx

# 예시 데이터 설정
data = {
    'High': [37385.35, 37376.7, 37361.29, 37353.66, 37352.50, 37351.36, 37362.59, 37398.19, 37412.65, 37408.11,
37431.65, 37431.10, 37429.15, 37418.81],
    'Low': [37361.87, 37360.21, 37346.27, 37336.73, 37333.99, 37351.06, 37344.18, 37354.74, 37396.75, 37397.0,
37402.44, 37411.75, 37415.41, 37392.31],
    'Close': [37376.16, 37360.41, 37352.95, 3733.57, 37351.36, 37361.68, 37355.97, 37398.19, 37407.35, 37402.82,
37416.33, 37428.85, 37418.81, 37394.27]
}

df = pd.DataFrame(data)

# ADX 계산
adx_result = calculate_adx(df['High'], df['Low'], df['Close'])

# 결과 출력
print("ADX 결과:")
print(adx_result)


ADX 결과:
0            NaN
1     100.000000
2     100.000000
3     100.000000
4     100.000000
5     100.000000
6      99.874183
7      99.247954
8      98.434075
9      97.728713
10     96.367708
11     95.188171
12     94.165905
13     93.384471
dtype: float64


In [9]:
def calculate_adx(high, low, close, period=14):
    tr = []  # True Range
    plus_dm = []  # Plus Directional Movement
    minus_dm = []  # Minus Directional Movement

    for i in range(1, len(close)):
        tr_value = max(high[i] - low[i], abs(high[i] - close[i - 1]), abs(low[i] - close[i - 1]))
        tr.append(tr_value)

        plus_dm_value = max(high[i] - high[i - 1], 0)
        plus_dm.append(plus_dm_value)

        minus_dm_value = max(low[i - 1] - low[i], 0)
        minus_dm.append(minus_dm_value)

    atr = [tr[0]]  # Average True Range
    plus_di = [plus_dm[0]]  # Plus Directional Indicator
    minus_di = [minus_dm[0]]  # Minus Directional Indicator

    for i in range(1, len(tr)):
        atr_value = ((period - 1) * atr[i - 1] + tr[i]) / period
        atr.append(atr_value)

        plus_di_value = ((period - 1) * plus_di[i - 1] + plus_dm[i]) / period
        plus_di.append(plus_di_value)

        minus_di_value = ((period - 1) * minus_di[i - 1] + minus_dm[i]) / period
        minus_di.append(minus_di_value)

    dx = [0]  # Directional Movement Index
    adx = [0]  # Average Directional Index

    for i in range(1, len(tr)):
        if plus_di[i] + minus_di[i] == 0:
            dx_value = 0
        else:
            dx_value = (abs(plus_di[i] - minus_di[i]) / (plus_di[i] + minus_di[i])) * 100

        dx.append(dx_value)

        adx_value = ((period - 1) * adx[i - 1] + dx[i]) / period
        adx.append(adx_value)

    return adx

# 예시 데이터 설정
high = [37385.35, 37376.7, 37361.29, 37353.66, 37352.50, 37351.36, 37362.59, 37398.19, 37412.65, 37408.11,
37431.65, 37431.10, 37429.15, 37418.81]
low = [37361.87, 37360.21, 37346.27, 37336.73, 37333.99, 37351.06, 37344.18, 37354.74, 37396.75, 37397.0,
37402.44, 37411.75, 37415.41, 37392.31]
close = [37376.16, 37360.41, 37352.95, 3733.57, 37351.36, 37361.68, 37355.97, 37398.19, 37407.35, 37402.82,
37416.33, 37428.85, 37418.81, 37394.27]

# ADX 계산
adx_result = calculate_adx(high, low, close)

# 결과 출력
print("ADX 결과:")
print(adx_result)


ADX 결과:
[0, 7.142857142857143, 13.775510204081632, 19.934402332361515, 25.653373594335694, 28.021214729972254, 26.50275535051474, 26.111536226767225, 25.748261326144533, 26.675065357736067, 27.535669101356778, 28.33480114900458, 26.94173472919004]


In [ ]:
    'High': ,
    'Low': [,
    'Close': 